Team Comparisons

In [64]:
import csv
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression

Import Data

In [65]:
Stats = []
with open('TI9.csv', newline = '') as csvfile:
    spamreader = csv.reader(csvfile, delimiter = ',', quotechar='|')
    for row in spamreader:
        Stats.append(row)

Games Array

In [66]:
Team1 = []
Team2 = []
Winner = []
i = 0
for game in Stats:
    if i == 0:
        i += 1
    else:
        Team1.append(Stats[i][1])
        Team2.append(Stats[i][4])
        Winner.append(Stats[i][7])
        i += 1       

Total Wins per team

In [67]:
Teamlist = ['OG','Team Liquid', 'PSG.LGD','Team Secret','Vici Gaming','Evil Geniuses','Royal Never Give Up','Infamous Gaming', 'Newbee', 'TNC Predator','Virtus.pro','Fnatic','Alliance','Mineski','Natus Vincere', 'Chaos Esports Club', 'KEEN GAMING', 'Ninjas in Pyjamas']


wij = np.zeros((len(Teamlist), len(Teamlist)))


wincounts = []
for x in Teamlist:
    wincounts.append(Winner.count(x))

Wins i vs j matrix

In [68]:
binWins = np.zeros(len(Winner))
for i in range(len(Winner)):
    if Winner[i] == Team1[i]:
        binWins[i] = 1

d = {'Radiant':Team1, 'Dire':Team2, 'Winner':binWins}

index_dict = {'OG':0,'Team Liquid':1,'PSG.LGD':2,'Team Secret':3,'Vici Gaming':4,'Evil Geniuses':5,'Royal Never Give Up':6,'Infamous Gaming':7,'Newbee':8,'TNC Predator':9,'Virtus.pro':10,'Fnatic':11,'Alliance':12,'Mineski':13,'Natus Vincere':14,'Chaos Esports Club':15,'KEEN GAMING':16,'Ninjas in Pyjamas':17}

games = pd.DataFrame(data = d)

Wi = pd.DataFrame(np.array([wincounts]), columns = Teamlist)
    
np.array(wincounts)


for n in range(len(games.loc[:,'Radiant'])):
    i = games.loc[n,'Radiant']
    j = games.loc[n,'Dire']
    if games.loc[n,'Winner'] == i:
        wij[index_dict[i],index_dict[j]] += 1
    else:
        wij[index_dict[j],index_dict[i]] += 1

To implement the Bradley Terry Model as a GLM we will be relying on the logistic model for binomially distributed data as implemented by Jamil, 2010 such that logit(p_(i>j))= log(p_(i>j)/(1-p_(i>j)) = lamda_i - lambda_j


Model Matrix

From here we should be able to apply the logit model as we have pij, X and unknowns lambda where
logit[pij] = X*Lambda

Our inputs are clearly 2 teams with an output vector as 0 or 1 depending whether team 1 or team 2 wins

Lets do some quick Data fixing on our existing design and response matrixes

In [69]:
"""Number of teams"""
k = len(Teamlist)
N = len(Winner)
X = np.zeros((N,k))
gameNumber = 0
for index, row in games.iterrows():
    X[gameNumber][index_dict[row['Radiant']]] = 1
    X[gameNumber][index_dict[row['Dire']]] = -1
    gameNumber+= 1
    

X = np.matrix(X)
Xt = np.matrix.transpose(X)
XXt = np.dot(X,Xt)
Y = np.matrix(binWins)
Y = np.matrix.transpose(Y)
rankings = np.matrix(np.random.rand(k,1))
tol = 1.

In [70]:
while tol > 0.00001:
    p = np.exp(np.dot(X,rankings))/(1+np.exp(np.dot(X,rankings)))
    gradient = np.matrix(np.zeros((1,k)))
    hessian = 0
    for i in range(N):
        gradient += np.multiply(X[i],Y[i]) - np.multiply(X[i],p[i])
        hessian += XXt.item((i,i))*p[i]*(1-p[i])\
        
    oldrankings = np.matrix.transpose(rankings)
    rankings = oldrankings + (1/hessian)*gradient
    
    tol = np.dot(rankings-oldrankings, np.matrix.transpose(rankings-oldrankings))
    rankings = np.matrix.transpose(rankings)

In [71]:

ranks = np.asarray(rankings)
ranks = np.ndarray.tolist(ranks)


data = {"Teams":Teamlist, "Ranks":ranks}
data
Teamskills = pd.DataFrame(data)
Teamskills

,Teams,Ranks
0,OG,[2.111570864460211]
1,Team Liquid,[1.0978410448271811]
2,PSG.LGD,[1.7718723313674376]
3,Team Secret,[1.1728111990547374]
4,Vici Gaming,[0.7299942982298468]
5,Evil Geniuses,[0.6107223237217839]
6,Royal Never Give Up,[-0.0227949931726601]
7,Infamous Gaming,[0.1241009925457263]
8,Newbee,[0.5214375081377614]
9,TNC Predator,[0.532681055859199]


Hero Ratings

In [72]:
heroes1 = []
heroes2 = []
i = 0
for game in Stats:
    if i == 0:
        i += 1
    else:
        heroes1.append(Stats[i][2])
        heroes2.append(Stats[i][5])
        i += 1   

In [73]:
heroes1

['Io  Tiny  Abaddon  Timbersaw  Gyrocopter',
 'Rubick  Tidehunter  Ogre Magi  Templar Assassin  Juggernaut',
 'Tidehunter  Templar Assassin  Shadow Shaman  Lifestealer  Enigma',
 'Tidehunter  Chen  Rubick  Templar Assassin  Meepo',
 'Elder Titan  Rubick  Faceless Void  Shadow Fiend  Dark Seer',
 'Elder Titan  Rubick  Sven  Dark Seer  Templar Assassin',
 'Tiny  Faceless Void  Mirana  Tinker  Shadow Demon',
 'Elder Titan  Rubick  Magnus  Wraith King  Bristleback',
 'Chen  Tidehunter  Earth Spirit  Leshrac  Meepo',
 'Chen  Alchemist  Tidehunter  Rubick  Legion Commander',
 'Shadow Demon  Sand King  Jakiro  Sven  Dragon Knight',
 "Elder Titan  Faceless Void  Pangolier  Nature's Prophet  Zeus",
 'Tiny  Omniknight  Windranger  Crystal Maiden  Bristleback',
 'Leshrac  Tidehunter  Lion  Earth Spirit  Medusa',
 'Elder Titan  Alchemist  Rubick  Ember Spirit  Dark Seer',
 'Mirana  Ogre Magi  Tidehunter  Faceless Void  Tinker',
 'Tiny  Faceless Void  Witch Doctor  Enigma  Abaddon',
 'Magnus  Shado

Firstly we need to count comparative wins separately, then we will compare them as a team

In [74]:
heroes1new = []
heroes2new = []
for i in heroes1:
    heroes1new.append(i.split("  "))

for i in heroes2:
    heroes2new.append(i.split("  "))

In [75]:
#Herodict goes here
l = 0
hdict = {}
for i in heroes1new:
    for j in i:
        if j in hdict:
            continue
        else:
            hdict[j] = l
            l += 1

for i in heroes2new:
    for j in i:
        if j in hdict:
            continue
        else:
            hdict[j] = l
            l += 1
hdict

{'Io': 0,
 'Tiny': 1,
 'Abaddon': 2,
 'Timbersaw': 3,
 'Gyrocopter': 4,
 'Rubick': 5,
 'Tidehunter': 6,
 'Ogre Magi': 7,
 'Templar Assassin': 8,
 'Juggernaut': 9,
 'Shadow Shaman': 10,
 'Lifestealer': 11,
 'Enigma': 12,
 'Chen': 13,
 'Meepo': 14,
 'Elder Titan': 15,
 'Faceless Void': 16,
 'Shadow Fiend': 17,
 'Dark Seer': 18,
 'Sven': 19,
 'Mirana': 20,
 'Tinker': 21,
 'Shadow Demon': 22,
 'Magnus': 23,
 'Wraith King': 24,
 'Bristleback': 25,
 'Earth Spirit': 26,
 'Leshrac': 27,
 'Alchemist': 28,
 'Legion Commander': 29,
 'Sand King': 30,
 'Jakiro': 31,
 'Dragon Knight': 32,
 'Pangolier': 33,
 "Nature's Prophet": 34,
 'Zeus': 35,
 'Omniknight': 36,
 'Windranger': 37,
 'Crystal Maiden': 38,
 'Lion': 39,
 'Medusa': 40,
 'Ember Spirit': 41,
 'Witch Doctor': 42,
 'Silencer': 43,
 'Phantom Lancer': 44,
 'Riki': 45,
 'Ancient Apparition': 46,
 'Arc Warden': 47,
 'Centaur Warrunner': 48,
 'Enchantress': 49,
 'Invoker': 50,
 'Earthshaker': 51,
 'Oracle': 52,
 'Terrorblade': 53,
 'Lina': 54,
 '

Now to try and break down all possible matchups (which is a fuckload)

In [76]:
Matchups1 = []
binWins2 = []
for i in heroes1new:
    for j in i:
        Matchups1.append(j)
        Matchups1.append(j)
        Matchups1.append(j)
        Matchups1.append(j)
        Matchups1.append(j)


Matchups2 = []
for i in heroes2new:
    for j in range(5):
        Matchups2.append(i[0])
        Matchups2.append(i[1])
        Matchups2.append(i[2])
        Matchups2.append(i[3])
        Matchups2.append(i[4])

    
for i in binWins:
    for j in range(25):
        binWins2.append(i)

d = {'Radiant':Matchups1, 'Dire':Matchups2, 'Winner':binWins2}
heroGames = pd.DataFrame(data = d)


In [77]:
"""Number of Heroes"""
k = len(hdict.keys())
""" Number of Comparisons """
N = len(binWins2)
""" Design Matrix """
X = np.zeros((N,k))
gameNumber = 0
for index, row in heroGames.iterrows():
    X[gameNumber][hdict[row['Radiant']]] = 1
    X[gameNumber][hdict[row['Dire']]] = -1
    gameNumber+= 1
    

X = np.matrix(X)
Xt = np.matrix.transpose(X)
XXt = np.dot(X,Xt)
Y = np.matrix(binWins2)
Y = np.matrix.transpose(Y)
rankings = np.matrix(np.random.rand(k,1))
tol = 1.

In [78]:
time = 0
while tol > 0.00001:
    print(time)
    time+=1
    p = np.exp(np.dot(X,rankings))/(1+np.exp(np.dot(X,rankings)))
    gradient = np.matrix(np.zeros((1,k)))
    hessian = 0
    for i in range(N):
        gradient += np.multiply(X[i],Y[i]) - np.multiply(X[i],p[i])
        hessian += XXt.item((i,i))*p[i]*(1-p[i])\
        
    oldrankings = np.matrix.transpose(rankings)
    rankings = oldrankings + (1/hessian)*gradient
    
    tol = np.dot(rankings-oldrankings, np.matrix.transpose(rankings-oldrankings))
    rankings = np.matrix.transpose(rankings)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [81]:
ranks = np.asarray(rankings)
ranks = np.ndarray.tolist(ranks)

heroes = []
for i in hdict.keys():
    heroes.append(i)
    
data = {"Heroes":heroes, "Ranks":ranks}
data
Heroskills = pd.DataFrame(data)
Heroskills

,Heroes,Ranks
0,Io,[1.9773114080008418]
1,Tiny,[0.991010749991192]
2,Abaddon,[1.4045109263766524]
3,Timbersaw,[0.48542991806805863]
4,Gyrocopter,[1.2754175465776034]
...,...,...
110,Spirit Breaker,[0.5840623009612215]
111,Night Stalker,[1.8283647466725306]
112,Bounty Hunter,[-0.7961911325573935]
113,Venomancer,[-0.9166465192011913]


Lets do some data fixing now on rankings and our two data frames

In [84]:
Heroskills.sort_values(by=['Ranks'], inplace=True, ascending=False)
Teamskills.sort_values(by=['Ranks'], inplace=True, ascending=False)

In [87]:
Teamskills

,Teams,Ranks
0,OG,[2.111570864460211]
2,PSG.LGD,[1.7718723313674376]
3,Team Secret,[1.1728111990547374]
1,Team Liquid,[1.0978410448271811]
4,Vici Gaming,[0.7299942982298468]
13,Mineski,[0.6111712936993904]
5,Evil Geniuses,[0.6107223237217839]
9,TNC Predator,[0.532681055859199]
8,Newbee,[0.5214375081377614]
12,Alliance,[0.40485002305247625]


In [92]:
Heroskills.loc[Heroskills['Heroes'] == "Meepo"]

,Heroes,Ranks
14,Meepo,[1.6413038315505477]


In [91]:
Heroskills

,Heroes,Ranks
75,Winter Wyvern,[2.822426523812675]
18,Dark Seer,[2.101446106429652]
21,Tinker,[2.01837889459694]
37,Windranger,[2.006802291835729]
0,Io,[1.9773114080008418]
...,...,...
85,Drow Ranger,[-1.2294687911170996]
35,Zeus,[-1.7089943224908943]
104,Disruptor,[-1.8993691464378877]
101,Vengeful Spirit,[-1.930231106363405]


How do we interpret the 2 ratings, for teams it is clear, but for heroes i imagine we can add up the skills of both teams and treat them as a single team, to test

for treating heroes as players in a team we refer back to Haziq

We shall first implement this for team OG, haziq does this for liverpool and we will implement this in a similar way, what is worth noting for the final potential model is that in the EPL model there are no shared players while in ours "players" will predominantly be shared

From Huang et al. 2006

In [93]:
""" Consider k individuals """
k = len(hdict.keys())
k

115

In [95]:
for i in heroes1new:
    i.sort()

In [103]:
for i in heroes1new:
    l = 0
    for j in heroes1new:
        if l == 0:
            continue
        elif i == j:
            print(True)
        l+=1
    for j in heroes2new:
        if i ==j:
            print(True)

In [107]:
ri = binWins
""" qi = the sum over all times j is in a team of the win probability (basically sum of wins)"""
herodf = pd.DataFrame(data = {"Radiant": heroes1new, "Dire": heroes2new, "Winner":binWins})

In [121]:
q = np.zeros(len(hdict.keys()))
qplus = np.zeros(len(hdict.keys()))
qminus = np.zeros(len(hdict.keys()))
game = 0
for hero in hdict.keys():
    for i in range(len(binWins)):
        if hero in herodf["Radiant"][i]:
            if herodf["Winner"][i] == 1:
                q[hdict[hero]] += 1
                qplus[hdict[hero]] += 1
        elif hero in herodf["Dire"][i]:
            if herodf["Winner"][i] == 0:
                q[hdict[hero]] += 1
                qminus[hdict[hero]] += 1
        else:
            continue

In [123]:
qminus

array([ 9., 15.,  4.,  2., 15., 12., 16., 13.,  6.,  2.,  4., 13.,  6.,
        8.,  1., 27.,  9.,  2.,  2., 10., 14.,  1., 25.,  5.,  7.,  4.,
        4., 13.,  7.,  3., 13.,  4.,  4.,  4.,  1.,  0.,  5.,  7., 10.,
        7.,  3.,  9.,  4.,  0.,  1.,  0.,  7.,  2., 10., 11.,  7., 11.,
        3.,  1.,  7.,  1.,  5.,  2., 14.,  4.,  0.,  4.,  0.,  0.,  4.,
        8.,  2.,  2.,  5.,  8.,  1.,  4.,  2.,  3.,  2.,  2.,  1.,  1.,
        4.,  1.,  1.,  8.,  2.,  0.,  3.,  0.,  2.,  4.,  4.,  0.,  4.,
        0.,  0.,  0.,  0.,  1.,  1.,  2.,  0.,  3.,  0.,  0.,  6.,  0.,
        0.,  1.,  5.,  0.,  1.,  0.,  1.,  1.,  0.,  0.,  0.])

In [131]:
k = len(hdict.keys())
m = len(binWins)

po = np.matrix(np.random.rand(k,1))
maxiter = 1000;
for t in range(maxiter):
    for i in range(k):
        pp = np.tile(np.transpose(p), (binWins.size,1))
        qp = sum(np.multiply(pp,))

Start with initial p_j^0 > 0, j = 1,...,k and obtain corresponding qi's values for i = 1,...,m
k = no.heros
m = no.games



In [132]:
pp

matrix([[0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727],
        [0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727],
        [0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727],
        ...,
        [0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727],
        [0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727],
        [0.55933094, 0.81263183, 0.7727807 , ..., 0.57746773, 0.43344969,
         0.41894727]])

In [126]:
m

193